In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the train and test CSV files
train = pd.read_csv('/content/drive/My Drive/train.csv')
test = pd.read_csv('/content/drive/My Drive/test.csv')
train=train[['text','ptext','index','label']]
test=test[['text','ptext','index','label']]
# Extract features (X) and labels (y) for train and test datasets
X_train = train[['text', 'ptext']]
y_train = train['label']

X_test = test[['text', 'ptext']]
y_test = test['label']


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame 'data' with a 'label_index' column containing label indices

# Split the data into training and test sets with stratified sampling
#X_training, X_validation, y_training, y_validation = train_test_split(
#    train[['text','ptext','index']],  # Features (TF-IDF vectors)
#    train[['label','index']],      # Target variable (label indices)
#    test_size=0.2,
#    stratify=train['label'],  # Ensure stratified sampling based on label indices
#    random_state=48  # Set a random seed for reproducibility
#)

# Merge X_train and y_train into the train DataFrame based on 'index' column
#training = X_training.merge(y_training )

# Merge X_test and y_test into the test DataFrame based on 'index' column
#valid = X_validation.merge(y_validation)

# Now, the 'train' and 'test' DataFrames are merged based on the 'index' column
#training.to_csv('/content/drive/MyDrive/training.csv')
#valid.to_csv('/content/drive/MyDrive/validation.csv')


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming you have already renamed the "medical_specialty" column to "label"

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit the encoder to your unique labels and transform the "label" column

# Load the train and test CSV files
train = pd.read_csv('/content/drive/My Drive/training.csv')
valid=pd.read_csv('/content/drive/My Drive/validation.csv')
test = pd.read_csv('/content/drive/My Drive/test.csv')
# List of categories
categories = [
    "Surgery",
    "Consult - History and Phy.",
    "Cardiovascular / Pulmonary",
    "Orthopedic",
    "Radiology",
    "General Medicine",
    "Gastroenterology",
    "Neurology",
    "SOAP / Chart / Progress Notes",
    "Obstetrics / Gynecology",
    "Urology",
    "Discharge Summary"
]
specialty_numbers=[38, 5, 3, 27, 33, 15, 14, 22, 35, 24, 39, 10]

# Iterate through the 'label' column of train, test, and valid dataframes
for df in [train, test, valid]:
    df['label'] = df['label'].apply(lambda x: x if x in specialty_numbers else 40)
    df['label'] = label_encoder.fit_transform(df['label'])

train=train[['text','ptext','index','label']]
valid=valid[['text','ptext','index','label']]
test=test[['text','ptext','index','label']]
# Extract features (X) and labels (y) for train and test datasets
X_train = train[['text', 'ptext']]
y_train = train['label']

X_valid=valid[['text', 'ptext']]
y_valid = valid['label']

X_test = test[['text', 'ptext']]
y_test = test['label']


In [ ]:
specialty_counts = valid['label'].value_counts()
len(specialty_counts)

13

# bert

In [4]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 34.9 MB/s eta 0:00:00


In [5]:
!pip install torch torchvision torchaudio

In [2]:
# Define your train_texts, train_labels, val_texts, and val_labels
train_texts = X_train['text'].tolist()
val_texts = X_valid['text'].tolist()
train_labels = y_train.tolist()
val_labels = y_valid.tolist()
# Define your train_texts, train_labels, val_texts, and val_labels
test_texts = X_test['text'].tolist()
test_labels = y_test.tolist()

# Bert

In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load the BERT model and tokenizer
model_name = 'bert-base-uncased'  # You can change this to another BERT variant if needed
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=13)
#model.resize_token_embeddings(len(tokenizer))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Tokenize and encode the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))


In [10]:
batch_size = 16  # You can adjust this based on your GPU memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [11]:

# Tokenize and encode the training and validation data
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)

# Create PyTorch datasets
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=batch_size)



In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1
!export TORCH_USE_CUDA_DSA=1


In [13]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5  # You can adjust the number of training epochs

total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

best_val_accuracy = 0.0  # Initialize the best validation accuracy
best_model = None  # Initialize a variable to store the best model


# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Save the current model as the best model
        best_model = model

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Loss: 93.57073199748993, Validation Accuracy: 0.37610062893081764
Epoch 1/5, Loss: 93.57073199748993, Validation Accuracy: 0.37610062893081764
Epoch 2/5, Loss: 83.13656485080719, Validation Accuracy: 0.38616352201257864
Epoch 2/5, Loss: 83.13656485080719, Validation Accuracy: 0.38616352201257864
Epoch 3/5, Loss: 77.87187707424164, Validation Accuracy: 0.37610062893081764
Epoch 3/5, Loss: 77.87187707424164, Validation Accuracy: 0.37610062893081764
Epoch 4/5, Loss: 77.00544857978821, Validation Accuracy: 0.37358490566037733
Epoch 4/5, Loss: 77.00544857978821, Validation Accuracy: 0.37358490566037733
Epoch 5/5, Loss: 75.6103504896164, Validation Accuracy: 0.379874213836478
Epoch 5/5, Loss: 75.6103504896164, Validation Accuracy: 0.379874213836478


In [14]:
# Evaluate the model on the test data
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})


In [15]:
from sklearn.metrics import accuracy_score

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 35.71%


In [16]:
# Create a DataFrame with a single column for predictions
test_predictions_df = pd.DataFrame({'predictions': test_predictions})

# Save the DataFrame to a CSV file
test_predictions_df.to_csv('/content/drive/MyDrive/bert_r.csv', index=False)

# another model

In [3]:
from sklearn.metrics import accuracy_score
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load the BERT model and tokenizer
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=13)
# Tokenize and encode the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))

batch_size = 16  # You can adjust this based on your GPU memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Tokenize and encode the training and validation data
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt', max_length=512)

# Create PyTorch datasets
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5  # You can adjust the number of training epochs

total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

best_val_accuracy = 0.0  # Initialize the best validation accuracy
best_model = None  # Initialize a variable to store the best model


# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Save the current model as the best model
        best_model = model

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate the model on the test data
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Loss: 85.93884408473969, Validation Accuracy: 0.379874213836478
Epoch 1/5, Loss: 85.93884408473969, Validation Accuracy: 0.379874213836478
Epoch 2/5, Loss: 73.86971783638, Validation Accuracy: 0.389937106918239
Epoch 2/5, Loss: 73.86971783638, Validation Accuracy: 0.389937106918239
Epoch 3/5, Loss: 70.2676203250885, Validation Accuracy: 0.3886792452830189
Epoch 3/5, Loss: 70.2676203250885, Validation Accuracy: 0.3886792452830189
Epoch 4/5, Loss: 68.48831903934479, Validation Accuracy: 0.3672955974842767
Epoch 4/5, Loss: 68.48831903934479, Validation Accuracy: 0.3672955974842767
Epoch 5/5, Loss: 67.61198878288269, Validation Accuracy: 0.3672955974842767
Epoch 5/5, Loss: 67.61198878288269, Validation Accuracy: 0.3672955974842767
Accuracy: 38.83%


In [4]:

# Evaluate the model on the test data
best_model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = best_model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 38.83%


In [5]:
# Create a DataFrame with a single column for predictions
test_predictions_df = pd.DataFrame({'predictions': test_predictions})

# Save the DataFrame to a CSV file
test_predictions_df.to_csv('/content/drive/MyDrive/Mbert_r.csv', index=False)

# gpt2

In [3]:
from sklearn.metrics import accuracy_score
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load the GPT-2 model and tokenizer
model_name = 'gpt2'  # You can use other variants like 'gpt2-medium', 'gpt2-large', etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=13)
# Define the padding token ID

batch_size=8
# Choose a fixed sequence length
max_length = 512
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'
pad_token_id = tokenizer.pad_token_id
model.config.pad_token_id = pad_token_id
model.resize_token_embeddings(len(tokenizer))

# Tokenize and encode the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))


train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5  # You can adjust the number of training epochs

total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
best_val_accuracy = 0.0  # Initialize the best validation accuracy
best_model = None  # Initialize a variable to store the best model


# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Save the current model as the best model
        best_model = model

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate the model on the test data
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  w

Epoch 1/5, Loss: 223.85664427280426, Validation Accuracy: 0.28679245283018867
Epoch 1/5, Loss: 223.85664427280426, Validation Accuracy: 0.28679245283018867
Epoch 2/5, Loss: 205.60119223594666, Validation Accuracy: 0.3283018867924528
Epoch 2/5, Loss: 205.60119223594666, Validation Accuracy: 0.3283018867924528
Epoch 3/5, Loss: 191.99288094043732, Validation Accuracy: 0.3584905660377358
Epoch 3/5, Loss: 191.99288094043732, Validation Accuracy: 0.3584905660377358
Epoch 4/5, Loss: 183.12648272514343, Validation Accuracy: 0.35094339622641507
Epoch 4/5, Loss: 183.12648272514343, Validation Accuracy: 0.35094339622641507
Epoch 5/5, Loss: 178.60005009174347, Validation Accuracy: 0.34968553459119495
Epoch 5/5, Loss: 178.60005009174347, Validation Accuracy: 0.34968553459119495
Accuracy: 35.92%


In [4]:
# Evaluate the model on the test data
best_model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = best_model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 35.92%


In [5]:
# Create a DataFrame with a single column for predictions
test_predictions_df = pd.DataFrame({'predictions': test_predictions})

# Save the DataFrame to a CSV file
test_predictions_df.to_csv('/content/drive/MyDrive/gpt2_r.csv', index=False)

# with preprocessing

In [3]:
# Define your train_texts, train_labels, val_texts, and val_labels
train_texts = X_train['ptext'].tolist()
val_texts = X_valid['ptext'].tolist()
train_labels = y_train.tolist()
val_labels = y_valid.tolist()
# Define your train_texts, train_labels, val_texts, and val_labels
test_texts = X_test['ptext'].tolist()
test_labels = y_test.tolist()

In [ ]:
from sklearn.metrics import accuracy_score
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load the GPT-2 model and tokenizer
model_name = 'gpt2'  # You can use other variants like 'gpt2-medium', 'gpt2-large', etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=13)
# Define the padding token ID

batch_size=8
# Choose a fixed sequence length
max_length = 512
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'
pad_token_id = tokenizer.pad_token_id
model.config.pad_token_id = pad_token_id
model.resize_token_embeddings(len(tokenizer))

# Tokenize and encode the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))


train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5  # You can adjust the number of training epochs

total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
best_val_accuracy = 0.0  # Initialize the best validation accuracy
best_model = None  # Initialize a variable to store the best model


# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Save the current model as the best model
        best_model = model

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate the model on the test data
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  w

Epoch 1/5, Loss: 213.20538938045502, Validation Accuracy: 0.27672955974842767
Epoch 1/5, Loss: 213.20538938045502, Validation Accuracy: 0.27672955974842767
Epoch 2/5, Loss: 194.54091250896454, Validation Accuracy: 0.32452830188679244
Epoch 2/5, Loss: 194.54091250896454, Validation Accuracy: 0.32452830188679244
Epoch 3/5, Loss: 180.080184340477, Validation Accuracy: 0.35094339622641507
Epoch 3/5, Loss: 180.080184340477, Validation Accuracy: 0.35094339622641507
Epoch 4/5, Loss: 170.30094826221466, Validation Accuracy: 0.35094339622641507
Epoch 4/5, Loss: 170.30094826221466, Validation Accuracy: 0.35094339622641507
Epoch 5/5, Loss: 169.30381572246552, Validation Accuracy: 0.35471698113207545
Epoch 5/5, Loss: 169.30381572246552, Validation Accuracy: 0.35471698113207545
Accuracy: 35.01%


In [ ]:
best_val_accuracy

0.35471698113207545

In [ ]:
# Evaluate the model on the test data
best_model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = best_model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 35.01%


In [ ]:
# Create a DataFrame with a single column for predictions
test_predictions_df = pd.DataFrame({'predictions': test_predictions})

# Save the DataFrame to a CSV file
test_predictions_df.to_csv('/content/drive/MyDrive/gpt2_pp_r.csv', index=False)

# bert

In [ ]:
from sklearn.metrics import accuracy_score
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=13)

batch_size=16
# Choose a fixed sequence length
max_length = 512
# Tokenize and encode the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))


train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5  # You can adjust the number of training epochs

total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
best_val_accuracy = 0.0  # Initialize the best validation accuracy
best_model = None  # Initialize a variable to store the best model


# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Save the current model as the best model
        best_model = model

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate the model on the test data
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5, Loss: 96.9234311580658, Validation Accuracy: 0.3522012578616352
Epoch 1/5, Loss: 96.9234311580658, Validation Accuracy: 0.3522012578616352
Epoch 2/5, Loss: 85.40887320041656, Validation Accuracy: 0.3660377358490566
Epoch 2/5, Loss: 85.40887320041656, Validation Accuracy: 0.3660377358490566
Epoch 3/5, Loss: 79.97498905658722, Validation Accuracy: 0.36981132075471695
Epoch 3/5, Loss: 79.97498905658722, Validation Accuracy: 0.36981132075471695
Epoch 4/5, Loss: 77.13205742835999, Validation Accuracy: 0.38238993710691827
Epoch 4/5, Loss: 77.13205742835999, Validation Accuracy: 0.38238993710691827
Epoch 5/5, Loss: 76.36067366600037, Validation Accuracy: 0.3949685534591195
Epoch 5/5, Loss: 76.36067366600037, Validation Accuracy: 0.3949685534591195
Accuracy: 35.92%


In [ ]:
best_val_accuracy

0.3949685534591195

In [ ]:
# Evaluate the model on the test data
best_model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = best_model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 35.92%


In [ ]:
# Create a DataFrame with a single column for predictions
test_predictions_df = pd.DataFrame({'predictions': test_predictions})

# Save the DataFrame to a CSV file
test_predictions_df.to_csv('/content/drive/MyDrive/bert_pp_r.csv', index=False)

# another bert

In [9]:
from sklearn.metrics import accuracy_score
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Load the BERT model and tokenizer
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=13)

batch_size=16
# Choose a fixed sequence length
max_length = 512
# Tokenize and encode the training and validation data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))


train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5  # You can adjust the number of training epochs

total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
best_val_accuracy = 0.0  # Initialize the best validation accuracy
best_model = None  # Initialize a variable to store the best model


# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

    # Check if the current model has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Save the current model as the best model
        best_model = model

    val_accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate the model on the test data
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Loss: 91.04285192489624, Validation Accuracy: 0.3622641509433962
Epoch 1/5, Loss: 91.04285192489624, Validation Accuracy: 0.3622641509433962
Epoch 2/5, Loss: 77.94576478004456, Validation Accuracy: 0.37610062893081764
Epoch 2/5, Loss: 77.94576478004456, Validation Accuracy: 0.37610062893081764
Epoch 3/5, Loss: 74.05330491065979, Validation Accuracy: 0.3836477987421384
Epoch 3/5, Loss: 74.05330491065979, Validation Accuracy: 0.3836477987421384
Epoch 4/5, Loss: 71.90336048603058, Validation Accuracy: 0.37610062893081764
Epoch 4/5, Loss: 71.90336048603058, Validation Accuracy: 0.37610062893081764
Epoch 5/5, Loss: 71.36797392368317, Validation Accuracy: 0.37358490566037733
Epoch 5/5, Loss: 71.36797392368317, Validation Accuracy: 0.37358490566037733
Accuracy: 40.85%


In [10]:
best_val_accuracy

0.3836477987421384

In [11]:
# Evaluate the model on the test data
best_model.eval()
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch  # Unpack all three elements
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = best_model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted.cpu().numpy())

# Convert the predictions to a DataFrame
test_results = pd.DataFrame({'label': test_predictions})

# Assuming you have 'test_labels' as true labels and 'test_predictions' as predicted labels
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 40.85%


In [12]:
# Create a DataFrame with a single column for predictions
test_predictions_df = pd.DataFrame({'predictions': test_predictions})

# Save the DataFrame to a CSV file
test_predictions_df.to_csv('/content/drive/MyDrive/Mbert_pp_r.csv', index=False)